In [1]:
from google.cloud import storage
from collections import OrderedDict
import os
import pandas as pd
from pandas.io.json import json_normalize
import json
import schedule
import time
import requests
import os

In [ ]:
def get_AQI_from_Air4Thai():
    
    url = "http://air4thai.pcd.go.th/services/getNewAQI_JSON.php"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.155 Safari/537.36',
    'cache-control': 'private, max-age=0, no-cache'}
    r = requests.get(url, headers=headers)
    
    if  r.status_code == 200:
        df = pd.read_json(url, orient='records')
        dp = pd.json_normalize(df['stations'])
        cols = dp.columns
        cols=cols.str.replace('LastUpdate.', '')
        cols=cols.str.replace('.value', '')
        dp.columns=cols
        dp.reset_index(inplace=False)
        d_out = dp[['stationID','areaEN','stationType','lat','long','date','time',
                    'PM25','PM10','CO','NO2','SO2','AQI']]
             
    else: 
        print("API Error")
       
    return d_out

start_time = time.time()
old_time = ''
old_date = ''
# begin loop here
while (True): 
    print(' ----- ' + 'Current time : ' + time.ctime() + ' -----') # to keep tracking running activity
    
    #Error handling for aqi scraping
    try:
        data = get_AQI_from_Air4Thai()
        data_time = data['time'][0]
        data_date = data['date'][0]
        print(data_time,data_date )
    except (RuntimeError, TypeError, NameError, ValueError):
        old_time = data_time # to keep forward
        print('error catched')
         
    if (data_time != old_time):  
        print(data)
        # if file does not exist write header 
        if not os.path.isfile(f'{data_date}--Air4Thai.csv'):
            data.to_csv(f'{data_date}--Air4Thai.csv',index =False)
        else: # else it exists so append without writing the header
            data.to_csv(f'{data_date}--Air4Thai.csv', mode='a', index =False, header=False)
        if(data_time == '23:00'):
            os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\NapatCota\Python\pm25-air4thai-45a8f7c75e1e.json'
            client = storage.Client()
            bucket = client.get_bucket('pm-25-bucket') 
            bucket.blob(f'Air4thai/{data_date}--pm25.csv').upload_from_filename(f'{data_date}--Air4Thai.csv', 'text/csv')
            
        old_time = data_time # Keep update old time
    else:
        print('No new update')
       
    
    time.sleep(900) # 900 seconds = 15 minutes

 ----- Current time : Mon Jul  5 15:25:02 2021 -----
15:00 2021-07-05
    stationID        date   time PM25 PM10
0         02t  2021-07-05  15:00    9   19
1         03t  2021-07-05  15:00   14   36
2         05t  2021-07-05  15:00    7   16
3         10t  2021-07-05  15:00    8   17
4         11t  2021-07-05  15:00    8   15
..        ...         ...    ...  ...  ...
125       35t  2021-07-05  15:00   13   24
126       36t  2021-07-05  15:00    7   19
127       72t  2021-07-05  15:00    8   17
128       69t  2021-07-05  15:00   11   19
129       58t  2021-07-05  15:00    4    9

[130 rows x 5 columns]
 ----- Current time : Mon Jul  5 15:40:02 2021 -----
15:00 2021-07-05
No new update
 ----- Current time : Mon Jul  5 15:55:02 2021 -----
15:00 2021-07-05
No new update
 ----- Current time : Mon Jul  5 16:10:03 2021 -----
16:00 2021-07-05
    stationID        date   time PM25 PM10
0         02t  2021-07-05  16:00    9   19
1         03t  2021-07-05  16:00   14   37
2         05t  2021-07-

In [ ]:
# Only need this if you're running this code locally.
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\NapatCota\Python\pm25-air4thai-45a8f7c75e1e.json'

df = get_AQI_from_Air4Thai()
client = storage.Client()
bucket = client.get_bucket('pm-25-bucket') 
  
bucket.blob('Air4thai/pm25.csv').upload_from_string(df.to_csv(), 'text/csv')

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\NapatCota\Python\pm25-air4thai-45a8f7c75e1e.json'
client = storage.Client()
bucket = client.get_bucket('pm-25-bucket') 
bucket.blob(f'Air4thai/20210701--pm25.csv').upload_from_filename(f'2021-07-01--Air4Thai.csv', 'text/csv')
